In [1]:
# Wczytanie danych
import pandas as pd

df_test = pd.read_csv("test.csv", sep=",", encoding="latin_1")
df = pd.read_csv("train.csv", sep=",", encoding="latin_1")

#display(df_test)

In [2]:
df_test.head()

,ItemID,SentimentText
0,1,is so sad for my APL frie...
1,2,I missed the New Moon trail...
2,3,omg its already 7:30 :O
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...


In [3]:
# Wybranie kolumn
df = df[["Sentiment","SentimentText"]]
df_test = df_test[["ItemID","SentimentText"]]

df = df.rename(columns={"SentimentText": "New"})

df.head()

,Sentiment,New
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [4]:
df_new = pd.concat([df_test, df], axis=1)
df_new = df_new.drop(columns=["New"])
df_new.head()

,ItemID,SentimentText,Sentiment
0,1,is so sad for my APL frie...,0.0
1,2,I missed the New Moon trail...,0.0
2,3,omg its already 7:30 :O,1.0
3,4,.. Omgaga. Im sooo im gunna CRy. I'...,0.0
4,5,i think mi bf is cheating on me!!! ...,0.0


In [5]:
# Przygotowanie danych
import re

def norm1(row):
    return re.sub("[-()\"#/@;:<>{}`+=~|.!?,]",'',row["SentimentText"])

df_new["norm"] = df_new.apply(norm1, axis=1)
df_new = df_new.drop("SentimentText", axis=1)
#display(df_new)

In [6]:
# Wektoryzacja
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
norm = df_new["norm"]
cv.fit(norm)
X = cv.transform(norm)

X_1 = X[0:99988]
y = df_new[['Sentiment']].iloc[0:99988]


X.shape

(299989, 284637)

In [7]:
# Wykorzystanie karty graficznej
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 


# Podział danych
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X_1, y)

num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [8]:
# Tworzenie sieci neuronowej Keras
from keras import models
from keras.layers import Dense, Dropout

model = models.Sequential()
model.add(Dense(100, activation="relu", input_shape=(284637,)))
model.add(Dropout(0.6))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.6))
model.add(Dense(num_classes, activation="sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               28463800  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 28,474,102
Trainable params: 28,474,102
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Kompilacja modelu
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [10]:
# Trenowanie modelu
results = model.fit(
    X_train, y_train,
    epochs = 2,
    batch_size = 512,
    validation_data = (X_test, y_test)
)

Train on 74991 samples, validate on 24997 samples
Epoch 1/2
74991/74991 [==============================] - 282s 4ms/step - loss: 0.5655 - acc: 0.7022 - val_loss: 0.4870 - val_acc: 0.7671
Epoch 2/2
74991/74991 [==============================] - 291s 4ms/step - loss: 0.4148 - acc: 0.8159 - val_loss: 0.4856 - val_acc: 0.7659


In [11]:
# Sprawdzenie skuteczności 
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Test loss:  0.48560370227005517
Test accuracy:  0.7658719046309399


In [12]:
# Predykcja danych testowych
results1 = model.predict(X, batch_size=512, verbose=0, steps=None)

In [13]:
# Stworzenie tablicy do podmiany warości już znanych
tablica = pd.read_csv("train.csv", sep=",", encoding="latin_1", header=0)
y_tablica = tablica["Sentiment"]
#y_tablica

In [14]:
# Przypisanie wyników do jedenj kolumny
a = results1.shape

predictions_cat = []
for i in range(0, a[0]):
    if results1[i, 0] < 0.5:
        predictions_cat.append(1)
    else:
        predictions_cat.append(0)

# Zastąpienie wartości znanych z pliku
predictions_to_file = predictions_cat
for i in range(0, 99989):
    predictions_to_file[i] = y_tablica[i]


file = 'wynik_2.csv'
#results_to_file = pd.DataFrame({'ItemID':df_new['ItemID'], 'Sentiment':predictions_cat})
results_to_file = pd.DataFrame({'ItemID':df_new['ItemID'], 'Sentiment':predictions_to_file})
results_to_file.to_csv(file, index=False)